In [1]:
import os
import random

import matplotlib.pyplot as plt
import numpy as np

In [2]:
import mr

In [3]:
from mrcnn import model as modellib
from mrcnn import visualize
from mrcnn import utils

Using TensorFlow backend.


In [10]:
work_path = os.path.join("E:", os.sep, "RCNNTanks256Valid")
os.chdir(work_path)

In [11]:
dataset = mr.MRDataset()
dataset.load(work_path)
dataset.prepare()

In [13]:
inference_config = mr.InferenceConfig()

# Recreate the model in inference mode
model = modellib.MaskRCNN(mode="inference",
                          config=inference_config,
                          model_dir="logs")

find_last = False
if find_last:
    model_path = model.find_last()
    print("Loading weights from ", model_path)
else:
    model_path = "mask_rcnn_mr_0316.h5"
model.load_weights(model_path, by_name=True)

In [15]:
def _comp_AP(image_id):
    image, image_meta, gt_class_id, gt_bbox, gt_mask = modellib.load_image_gt(dataset,
                                                                              inference_config,
                                                                              image_id,
                                                                              use_mini_mask=False)
    results = model.detect([image], verbose=0)
    r = results[0]
    AP, precisions, recalls, overlaps = utils.compute_ap(gt_bbox, gt_class_id, gt_mask,
                                                         r["rois"], r["class_ids"], r["scores"], r['masks'])
    return AP
    
image_ids = random.choices(dataset.image_ids, k=100)
APs = [_comp_AP(image_id) for image_id in image_ids]
print("mAP: ", np.mean(APs))

mAP:  0.7107162719610192
